In [1]:
import os

In [2]:
os.chdir('/Documents/CBA_Stats')

In [3]:
!pwd

/Documents/CBA_Stats


In [4]:
from cbastats import Scraper

In [6]:
sina_scraper = Scraper.SinaScraper(Scraper.SINA_SCHEDULE_BASE_URL,Scraper.ENCODING, Scraper.PARSER, Scraper.HEADERS)

In [7]:
sina_scraper.scrape_sina(season='20-21')

----------------------------------------
Composing url
----------------------------------------
Scraping Schedule
----------------------------------------
Insert Schedule to CBA_Staging.Schedules
----------------------------------------
Operation complete. Success is not garanteed. Check query to verify.
Cleaning up CBA_Staging.Schedules
----------------------------------------
Pulling schedule from CBA_Staging.Schedules
----------------------------------------


  0%|          | 0/20 [00:00<?, ?it/s]

Scraping game stats
----------------------------------------


100%|██████████| 20/20 [01:03<00:00,  3.19s/it]


Append game stats to CBA_Data.PlayerStatsPerGame
----------------------------------------
Operation complete. Success is not garanteed. Check row count to verify.
Final clean up of the staging schedules
----------------------------------------
